# Processing thread for XMM-Newton EPIC observations

## Calcluate the average RAWY for each CCD

The following is happening here:

1. Sets the folders where the ODF is and where the products will be saved
2. Checks if the necessary SAS files are available: ccf.cif and sets some environmental variables
3. Generates a spectrum per CCD

In [1]:
import os
import subprocess
import sys
import logging
import glob

import numpy as np

from astropy.io import fits

In [2]:
# function to run shell process
# get the SAS version, will be used for the output folder 
def run_command(command):
    try:
        result = subprocess.run([command], shell=True,stdout=subprocess.PIPE,stderr=subprocess.STDOUT)
        retcode = result.returncode
        if retcode < 0:
            print("Warning: \"{}\" was terminated by signal {} \n {}".format(command,-retcode,result.stdout.decode()))
            logging.warning("\"{}\" was terminated by signal {} \n {}".format(command,-retcode,result.stdout.decode()))
        else:
            print("Info: \"{}\" returned {} \n {}".format(command,retcode,result.stdout.decode()))
            logging.info("\"{}\" returned {} \n {}".format(command,retcode,result.stdout.decode()))
    except OSError as e:
        print("Execution of \"{}\" failed:".format(command), e, file=sys.stderr)
        logging.error("Execution of \"{}\" failed:".format(command), e, file=sys.stderr)
    return retcode

In [3]:
#obsid = "0721010501"
#obsid = "0721010401"
obsid = "0764530101"

home = os.path.expanduser('~')
wdir = f"{home}/IVAN/Cu-line"

if (not os.path.isdir(wdir)):
    print (f"The input working folder {wdir} does not exist.")
    raise FileNotFoundError
#
# the ODF directory
odfDir = os.path.join(wdir,obsid)
if (not os.path.isdir(odfDir)):
    print (f"The ODF folder {odfDir} does not exist! Cannot continue!")
    raise FileNotFoundError
#
ppsDir = os.path.join(odfDir,"PN_LW")
#
if (not os.path.isdir(ppsDir)):
    print (f"PPS folder {ppsDir} does not exist. You need to run step02 before running step03")
    raise FileNotFoundError
#
os.chdir(ppsDir)

In [4]:
os.environ['SAS_CCFPATH'] = '/xdata/ccf/pub'
os.environ["SAS_ODF"] = odfDir
ccffile = os.path.join(odfDir,"ccf.cif")
if (not os.path.isfile(ccffile)):
    print (f"No ccf.cif file found in ODF folder {odfDir}")
    print ("     ===> This means epic_proc_step01 was not run?")
    raise FileNotFoundError
#
os.environ["SAS_CCF"] = ccffile
#

In [5]:
# find the event lists
haveEvlist = False
evfind = glob.glob(f"{ppsDir}/*PIEVLI*")
if (len(evfind) >= 1):
    logging.info(f"Found {len(evfind)} event lists")
    # exposures
    xexpo = [os.path.basename(x)[13:17] for x in evfind]
    print ("Event lists for the following exposures are available: ",xexpo)
    haveEvlist = True
else:
    raise FileNotFoundError
#
index = 0
evlist = evfind[index]
nexpo = os.path.basename(evlist)[13:17]
print (f"Will process event list {evlist}")
#

Event lists for the following exposures are available:  ['S003']
Will process event list /home/ivaltchanov/IVAN/Cu-line/0764530101/PN_LW/P0764530101PNS003PIEVLI0000.FIT


In [16]:
print ("*** Generating a new event list")
#
evsel = "(PATTERN==0) && (PAT_SEQ==0) && #XMMEA_EP && (PI <= 9000) && (PI >= 7000)"
#
# now loop over all 12 CCDs in EPN
#
fout = open(f"{wdir}/{obsid}_rawy.csv","w")
print ("obsid,ccd,rawy,nevents",file=fout)
#
for i in range(12):
    ccd = f"{i+1:02}"
    print (f"Processing CCD # {ccd}")
    #
    mnk_mask =  f"/xdata/xcaldata/XMM/PN/CTI/mask/refmask_{ccd}_mnk_lmap.fits"
    #
    sel1 = f"mask({mnk_mask},1,1,RAWX,RAWY) && (CCDNR == {i+1})"
    new_events = f"pn_{nexpo}_{ccd}_events.fits"
    ev_comm = f'evselect table={evlist} withfilteredset=Y filteredset={new_events}' +  \
        f' destruct=Y keepfilteroutput=T expression=\'{evsel} && {sel1}\''
    status = run_command(ev_comm)
    if (status != 0):
        raise Exception
    #
    hdu = fits.open(new_events)
    #
    # now select the events in [7,9] keV
    rawy = hdu['EVENTS'].data['RAWY']
    nev = len(rawy)
    mean_rawy = np.mean(rawy)
    print (f"{obsid},{ccd},{mean_rawy},{nev}",file=fout)
    hdu.close()
#
print ("pn_lw_step04 done")
#

*** Generating a new event list
Processing CCD # 01
Info: "evselect table=/home/ivaltchanov/IVAN/Cu-line/0764530101/PN_LW/P0764530101PNS003PIEVLI0000.FIT withfilteredset=Y filteredset=pn_S003_01_events.fits destruct=Y keepfilteroutput=T expression='(PATTERN==0) && (PAT_SEQ==0) && #XMMEA_EP && (PI <= 9000) && (PI >= 7000) && mask(/xdata/xcaldata/XMM/PN/CTI/mask/refmask_01_mnk_lmap.fits,1,1,RAWX,RAWY) && (CCDNR == 1)'" returned 0 
 evselect:- Executing (routine): evselect table=/home/ivaltchanov/IVAN/Cu-line/0764530101/PN_LW/P0764530101PNS003PIEVLI0000.FIT filteredset=pn_S003_01_events.fits withfilteredset=yes keepfilteroutput=yes flagcolumn=EVFLAG flagbit=-1 destruct=yes dssblock='' expression='(PATTERN==0) && (PAT_SEQ==0) && #XMMEA_EP && (PI <= 9000) && (PI >= 7000) && mask(/xdata/xcaldata/XMM/PN/CTI/mask/refmask_01_mnk_lmap.fits,1,1,RAWX,RAWY) && (CCDNR == 1)' filtertype=expression cleandss=no updateexposure=yes filterexposure=yes writedss=yes blockstocopy='' attributestocopy='' energ

In [17]:
fout.close()